In [8]:
import torch
print(torch.__version__)
print("GPU available:", torch.cuda.is_available())


2.7.1+cpu
GPU available: False


Import Libraries

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
!pip install piq
!pip install torchmetrics
from piq import ssim, ms_ssim, psnr
from torchmetrics import (
    PeakSignalNoiseRatio,
    StructuralSimilarityIndexMeasure,
    MeanSquaredError,
    MultiScaleStructuralSimilarityIndexMeasure,
)
from torch.utils.data import DataLoader

import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import torchvision.transforms.functional as TF

'pip' is not recognized as an internal or external command,
operable program or batch file.
'pip' is not recognized as an internal or external command,
operable program or batch file.


Kodak Dataset

In [14]:
import os
import urllib.request
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# Kodak datasetini indir
def download_kodak_dataset(dest_dir='./kodak'):
    os.makedirs(dest_dir, exist_ok=True)
    for i in range(1, 25):
        filename = f'kodim{str(i).zfill(2)}.png'
        url = f"http://r0k.us/graphics/kodak/kodak/{filename}"
        dest = os.path.join(dest_dir, filename)
        if not os.path.exists(dest):
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(url, dest)
        else:
            print(f"{filename} already exists. Skipping download.")

# Kodak Dataset sınıfı
class KodakDataset(Dataset):
    def __init__(self, img_dir, transform=None):
        super().__init__()
        self.transform = transform
        self.img_paths = [os.path.join(img_dir, f) for f in sorted(os.listdir(img_dir))
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, idx):
        img = Image.open(self.img_paths[idx]).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, 0
    
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
transform1 = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
     transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)

])


download_kodak_dataset('./kodak')
kodak_dataset = KodakDataset('./kodak', transform=transform)
kodak_dataset1 = KodakDataset('./kodak', transform=transform)

DataLoaders

In [15]:

kodak_loader = torch.utils.data.DataLoader(kodak_dataset, batch_size=4, shuffle=True)
kodak_loader1 = torch.utils.data.DataLoader(kodak_dataset1, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)


for img, label in kodak_loader:
  print(img.shape)



print(f"Kodak datasetinde {len(kodak_dataset)} resim var.")
print(f"DataLoader batch sayısı: {len(kodak_loader)}")
print(f"DataLoader batch sayısı: {len(kodak_loader1)}")

print("Kodak size:", len(kodak_dataset))

torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
torch.Size([4, 3, 256, 256])
Kodak datasetinde 24 resim var.
DataLoader batch sayısı: 6
DataLoader batch sayısı: 1
Kodak size: 24


Loss Functions

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mse_metric = MeanSquaredError().to(device)
psnr_metric = PeakSignalNoiseRatio(data_range=1.0).to(device)
ssim_metric = StructuralSimilarityIndexMeasure(data_range=1.0).to(device)
ms_ssim_metric = MultiScaleStructuralSimilarityIndexMeasure(data_range=1.0).to(device)

def mse_loss(output, target):
    return mse_metric(output, target)

def psnr_loss(output, target):
    psnr_val = psnr_metric(output, target)
    psnr_norm = psnr_val / 50.0
    return 1 - psnr_norm

def ssim_loss(output, target):
    return 1 - ssim_metric(output, target)


def combined_loss(output, target, alpha=0.5):
    # alpha ile MSE ve 1-SSIM ağırlıklandırılıyor
    mse = mse_loss(output, target)
    ssim_val = ssim(output, target)
    # Not: SSIM yüksekse iyi, ama loss için 1-SSIM kullanılır
    return alpha * mse + (1 - alpha) * (1 - ssim_val)

Compression Rate

In [18]:
def compression_ratio(original_img, latent_tensor, quantization_levels=16):
    B, C, H, W = original_img.shape
    original_size_bits = B * C * H * W * 32  # float32 = 32 bit

    B_l, C_l, H_l, W_l = latent_tensor.shape
    bits_per_value = int(torch.log2(torch.tensor(quantization_levels)).item())  # örn. 4 bit
    compressed_size_bits = B_l * C_l * H_l * W_l * bits_per_value

    return original_size_bits / compressed_size_bits

Basic Autoencoder

In [19]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent

Latent Quantizer Function

In [20]:
def quantize(tensor, levels=16):
    min_val = tensor.min()
    max_val = tensor.max()
    scale = (max_val - min_val) / (levels - 1)
    return torch.round((tensor - min_val) / scale) * scale + min_val

Quantized Autoencoder

In [21]:
class AutoencoderQuantized(nn.Module):
    def __init__(self, quantization_levels=64):
        super(AutoencoderQuantized, self).__init__()
        self.quantization_levels = quantization_levels  # Burada tanımla
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, stride=2, padding=1),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)

        scale = self.quantization_levels - 1
        quantized_latent = torch.round(latent * scale) / scale

        reconstructed = self.decoder(quantized_latent)
        return reconstructed, quantized_latent


Improved Autoencoder

In [22]:
class ImprovedAutoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 64, 4, stride=2, padding=1),  # 64x H/2 x W/2
            nn.ReLU(),
            nn.Conv2d(64, 128, 4, stride=2, padding=1), # 128x H/4 x W/4
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.Conv2d(128, 256, 4, stride=2, padding=1), # 256x H/8 x W/8
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 4, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 3, 4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed, latent


Model, Loss & Optimizer Definition

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Autoencoder().to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


VGG and perceptual Loss

In [24]:
import torch
import torch.nn as nn
import torchvision.models as models

class VGGFeatureExtractor(nn.Module):
    def __init__(self, layers=['3', '8', '15', '22']):
        super().__init__()
        vgg_pretrained = models.vgg16(pretrained=True).features
        self.selected_layers = layers
        self.layers = nn.ModuleList([vgg_pretrained[i] for i in range(max(map(int, layers)) + 1)])
        for param in self.parameters():
            param.requires_grad = False

    def forward(self, x):
        features = []
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if str(i) in self.selected_layers:
                features.append(x)
        return features


def perceptual_loss(output, target, feature_extractor, criterion=nn.L1Loss()):
    def normalize_for_vgg(x):
        mean = torch.tensor([0.485, 0.456, 0.406], device=x.device).view(1, 3, 1, 1)
        std = torch.tensor([0.229, 0.224, 0.225], device=x.device).view(1, 3, 1, 1)
        return (x - mean) / std

    output = normalize_for_vgg(output)
    target = normalize_for_vgg(target)

    output_features = feature_extractor(output)
    target_features = feature_extractor(target)
    loss = 0
    for of, tf in zip(output_features, target_features):
        loss += criterion(of, tf)
    return loss


import torch
import torch.nn as nn

def perc_train_model(model, dataloader, optimizer, num_epochs, device):
    mse_criterion = nn.MSELoss()
    vgg_extractor = VGGFeatureExtractor().to(device)
    vgg_extractor.eval()  # Sadece feature extraction, eğitim yok

    alpha = 0.85  # MSE loss ağırlığı
    beta = 0.15   # Perceptual loss ağırlığı

    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_recon_loss = 0.0
        running_perceptual_loss = 0.0

        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            targets = inputs.clone()


            optimizer.zero_grad()
            outputs, _ = model(inputs)

            # MSE loss
            recon_loss = mse_criterion(outputs, targets)

            # Perceptual loss
            p_loss = perceptual_loss(outputs, targets, vgg_extractor)

            # Toplam loss
            loss = alpha * recon_loss + beta * p_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_recon_loss += recon_loss.item() * inputs.size(0)
            running_perceptual_loss += p_loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        epoch_recon_loss = running_recon_loss / len(dataloader.dataset)
        epoch_p_loss = running_perceptual_loss / len(dataloader.dataset)

        print(f"Epoch [{epoch+1}/{num_epochs}], Total Loss: {epoch_loss:.4f}, Recon Loss: {epoch_recon_loss:.4f}, Perceptual Loss: {epoch_p_loss:.4f}")

import matplotlib.pyplot as plt

def reconstruct_and_visualize(model, dataloader, device, num_images=5, title="Reconstruction"):
    model.eval()
    model.to(device)
    images_shown = 0

    with torch.no_grad():
        for inputs in dataloader:
            if isinstance(inputs, (list, tuple)) and len(inputs) == 2:
                inputs = inputs[0]

            inputs = inputs.to(device)
            outputs, _ = model(inputs)

            inputs = inputs.cpu()
            outputs = outputs.cpu()

            for i in range(min(num_images, inputs.size(0))):
                input_img = inputs[i].permute(1, 2, 0).clamp(0, 1)
                output_img = outputs[i].permute(1, 2, 0).clamp(0, 1)

                fig, axes = plt.subplots(1, 2, figsize=(8, 4))
                axes[0].imshow(input_img.numpy())
                axes[0].set_title("Original")
                axes[0].axis("off")

                axes[1].imshow(output_img.numpy())
                axes[1].set_title("Reconstructed")
                axes[1].axis("off")

                fig.suptitle(title)
                plt.tight_layout()
                plt.show()

                images_shown += 1
                if images_shown >= num_images:
                    return

Train Function

In [26]:
def train_model(model, loss_fn, optimizer, train_loader, num_epochs=10, loss_name="Loss", save_path="autoencoder.pth"):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0.0
        epoch_compression = 0.0
        total_batches = 0

        for data in train_loader:
            imgs, _ = data
            imgs = imgs.to(device)

            # Modelden hem çıktı hem de latent temsil alınmalı
            outputs, latent = model(imgs)
            loss = loss_fn(outputs, imgs)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            total_batches += 1

            # Compression ratio hesapla
            cr = compression_ratio(imgs, latent, quantization_levels=16)
            epoch_compression += cr

        avg_loss = epoch_loss / total_batches
        avg_compression = epoch_compression / total_batches

        print(f"Epoch [{epoch+1}/{num_epochs}], {loss_name}: {avg_loss:.4f}, Compression Ratio: {avg_compression:.2f}")

        torch.save(model.state_dict(), save_path)

    print("-----------------------------------------------")

Test & Visualize Function

In [16]:
import matplotlib.pyplot as plt
import torch

def test_and_visualize_model(model, weight_path, test_loader, title="", device='cuda',
                             max_batches=5, max_images_per_batch=4, output_color_space="rgb"):
    """
    Modeli test eder, batch batch çıktı üretir ve görselleştirir.
    output_color_space parametresi ile çıktı rgb veya yuv olabilir.

    Args:
        model (nn.Module): Model nesnesi
        weight_path (str): Model ağırlıklarının yolu
        test_loader (DataLoader): Test veri yükleyicisi
        title (str): Görselleştirme başlığı
        device (str): 'cuda' ya da 'cpu'
        max_batches (int): En fazla kaç batch gösterilecek (default 5)
        max_images_per_batch (int): Her batch'ten kaç resim gösterilecek (default 4)
        output_color_space (str): 'rgb' veya 'yuv' (default 'rgb')
    """
    model.load_state_dict(torch.load(weight_path))
    model.to(device)
    model.eval()

    with torch.no_grad():
        for batch_idx, (imgs, _) in enumerate(test_loader):
            imgs = imgs.to(device)
            outputs, _ = model(imgs)

            # Eğer çıktı YCbCr ise, RGB'ye çevir
            if output_color_space.lower() == "ycbcr":
                outputs = ycbcr_to_rgb(outputs)

            batch_size = imgs.size(0)
            n = min(batch_size, max_images_per_batch)

            fig, axes = plt.subplots(2, n, figsize=(3 * n, 6))
            if n == 1:
                axes = axes.reshape(2, 1)  # axes'leri 2D array gibi kullanmak için

            for i in range(n):
                # Orijinal
                axes[0, i].imshow(imgs[i].cpu().permute(1, 2, 0).clamp(0, 1))
                axes[0, i].set_title("Original")
                axes[0, i].axis('off')

                # Rekonstrüksiyon
                axes[1, i].imshow(outputs[i].cpu().permute(1, 2, 0).clamp(0, 1))
                axes[1, i].set_title("Reconstructed")
                axes[1, i].axis('off')

            plt.suptitle(f"{title} - Batch {batch_idx + 1}")
            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.show()

            if batch_idx + 1 >= max_batches:
                break


JPEG

In [27]:
import io

def jpeg_compress(img_tensor, quality=25):
    img_pil = TF.to_pil_image(img_tensor.cpu())
    buffer = io.BytesIO()
    img_pil.save(buffer, format="JPEG", quality=quality)
    buffer.seek(0)
    compressed = Image.open(buffer)
    return TF.to_tensor(compressed).to(device)

def test_jpeg_quality(test_loader, quality=25):
    mse_metric = MeanSquaredError().to(device)
    psnr_metric = PeakSignalNoiseRatio().to(device)
    ssim_metric = StructuralSimilarityIndexMeasure().to(device)

    with torch.no_grad():
        for data in test_loader:
            imgs, _ = data
            imgs = imgs.to(device)
            compressed_imgs = torch.stack([jpeg_compress(im, quality) for im in imgs])

            mse_val = mse_metric(compressed_imgs, imgs).item()
            psnr_val = psnr_metric(compressed_imgs, imgs).item()
            ssim_val = ssim_metric(compressed_imgs, imgs).item()

            print(f"JPEG Quality {quality} - MSE: {mse_val:.6f}, PSNR: {psnr_val:.2f}, SSIM: {ssim_val:.4f}")
            break

Datasets Info

In [28]:
datasets_info = [
    {
        "name": "Kodak",
        "train_loader": kodak_loader,
        "test_loader": kodak_loader,
    },
]

Training with Autoencoder

In [29]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start Autoencoder
    model = Autoencoder().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Eğitim: MSE Loss
    print(f"Training {ds['name']} Autoencoder with MSE Loss...")
    model_path = f"{ds['name'].lower()}_autoencoder_mse.pth"
    train_model(
        model=model,
        loss_fn=mse_loss,
        optimizer=optimizer,
        train_loader=train_loader,
        num_epochs=10,
        loss_name="MSE Loss",
        save_path=model_path
    )

    print(f"Testing {ds['name']} Autoencoder...")
    # test_and_visualize_model(model, weight_path=model_path, test_loader=test_loader, title=f"{ds['name']} Reconstructed (MSE)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=25)



--- Dataset: Kodak ---
Training Kodak Autoencoder with MSE Loss...
Epoch [1/10], MSE Loss: 0.0523, Compression Ratio: 12.00
Epoch [2/10], MSE Loss: 0.0514, Compression Ratio: 12.00
Epoch [3/10], MSE Loss: 0.0501, Compression Ratio: 12.00
Epoch [4/10], MSE Loss: 0.0475, Compression Ratio: 12.00
Epoch [5/10], MSE Loss: 0.0433, Compression Ratio: 12.00
Epoch [6/10], MSE Loss: 0.0353, Compression Ratio: 12.00
Epoch [7/10], MSE Loss: 0.0252, Compression Ratio: 12.00
Epoch [8/10], MSE Loss: 0.0177, Compression Ratio: 12.00
Epoch [9/10], MSE Loss: 0.0142, Compression Ratio: 12.00
Epoch [10/10], MSE Loss: 0.0121, Compression Ratio: 12.00
-----------------------------------------------
Testing Kodak Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 25 - MSE: 0.000930, PSNR: 30.31, SSIM: 0.8590


In [30]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]


    # Start Autoencoder
    model = Autoencoder().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Training: PSNR Loss

    print(f"Training {ds['name']} Autoencoder with PSNR Loss...")
    train_model(model, psnr_loss, optimizer, train_loader, num_epochs=10, loss_name="PSNR Loss", save_path="autoencoder_psnr.pth")

    print(f"Testing {ds['name']} Autoencoder...")
    #test_and_visualize_model(model, weight_path="autoencoder_psnr.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (PSNR)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=25)


--- Dataset: Kodak ---
Training Kodak Autoencoder with PSNR Loss...
Epoch [1/10], PSNR Loss: 0.7500, Compression Ratio: 12.00
Epoch [2/10], PSNR Loss: 0.7483, Compression Ratio: 12.00
Epoch [3/10], PSNR Loss: 0.7453, Compression Ratio: 12.00
Epoch [4/10], PSNR Loss: 0.7413, Compression Ratio: 12.00
Epoch [5/10], PSNR Loss: 0.7339, Compression Ratio: 12.00
Epoch [6/10], PSNR Loss: 0.7255, Compression Ratio: 12.00
Epoch [7/10], PSNR Loss: 0.7085, Compression Ratio: 12.00
Epoch [8/10], PSNR Loss: 0.6812, Compression Ratio: 12.00
Epoch [9/10], PSNR Loss: 0.6481, Compression Ratio: 12.00
Epoch [10/10], PSNR Loss: 0.6352, Compression Ratio: 12.00
-----------------------------------------------
Testing Kodak Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 25 - MSE: 0.001022, PSNR: 29.91, SSIM: 0.8375


In [31]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]


    # Start Autoencoder
    model = Autoencoder().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    # Training:  SSIM Loss
    print(f"Training {ds['name']} Autoencoder with SSIM Loss...")
    train_model(model, ssim_loss, optimizer, train_loader, num_epochs=10, loss_name="SSIM Loss", save_path="autoencoder_ssim.pth")

    print(f"Testing {ds['name']} Autoencoder...")
    #test_and_visualize_model(model, weight_path="autoencoder_ssim.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (SSIM)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=25)




--- Dataset: Kodak ---
Training Kodak Autoencoder with SSIM Loss...
Epoch [1/10], SSIM Loss: 0.5796, Compression Ratio: 12.00
Epoch [2/10], SSIM Loss: 0.5772, Compression Ratio: 12.00
Epoch [3/10], SSIM Loss: 0.5768, Compression Ratio: 12.00
Epoch [4/10], SSIM Loss: 0.5764, Compression Ratio: 12.00
Epoch [5/10], SSIM Loss: 0.5755, Compression Ratio: 12.00
Epoch [6/10], SSIM Loss: 0.5734, Compression Ratio: 12.00
Epoch [7/10], SSIM Loss: 0.5686, Compression Ratio: 12.00
Epoch [8/10], SSIM Loss: 0.5579, Compression Ratio: 12.00
Epoch [9/10], SSIM Loss: 0.5367, Compression Ratio: 12.00
Epoch [10/10], SSIM Loss: 0.5037, Compression Ratio: 12.00
-----------------------------------------------
Testing Kodak Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 25 - MSE: 0.000959, PSNR: 30.18, SSIM: 0.8639


Training with Quantized Autoencoder

In [32]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start AutoencoderQuantized
    model1 = AutoencoderQuantized(quantization_levels=16).to(device)
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-3)

    # Training: MSE Loss
    print(f"Training {ds['name']} Quantized Autoencoder with MSE Loss...")
    train_model(model1, mse_loss, optimizer1, train_loader, num_epochs=10, loss_name="Quantized MSE Loss", save_path="autoencoder_mse_quantized.pth")

    print(f"Testing {ds['name']} Quantized Autoencoder...")
    #test_and_visualize_model(model1, weight_path="autoencoder_mse_quantized.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (MSE + Quant)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)


--- Dataset: Kodak ---
Training Kodak Quantized Autoencoder with MSE Loss...
Epoch [1/10], Quantized MSE Loss: 0.0523, Compression Ratio: 12.00
Epoch [2/10], Quantized MSE Loss: 0.0516, Compression Ratio: 12.00
Epoch [3/10], Quantized MSE Loss: 0.0510, Compression Ratio: 12.00
Epoch [4/10], Quantized MSE Loss: 0.0502, Compression Ratio: 12.00
Epoch [5/10], Quantized MSE Loss: 0.0493, Compression Ratio: 12.00
Epoch [6/10], Quantized MSE Loss: 0.0483, Compression Ratio: 12.00
Epoch [7/10], Quantized MSE Loss: 0.0471, Compression Ratio: 12.00
Epoch [8/10], Quantized MSE Loss: 0.0459, Compression Ratio: 12.00
Epoch [9/10], Quantized MSE Loss: 0.0444, Compression Ratio: 12.00
Epoch [10/10], Quantized MSE Loss: 0.0430, Compression Ratio: 12.00
-----------------------------------------------
Testing Kodak Quantized Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.007369, PSNR: 21.33, SSIM: 0.5339


In [33]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start AutoencoderQuantized
    model1 = AutoencoderQuantized(quantization_levels=16).to(device)
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-3)

    # Training: PSNR Loss
    print(f"Training {ds['name']} Quantized Autoencoder with PSNR Loss...")
    train_model(model1, psnr_loss, optimizer1, train_loader, num_epochs=10, loss_name="Quantized PSNR Loss", save_path="autoencoder_psnr_quantized.pth")

    print(f"Testing {ds['name']} Quantized Autoencoder...")
    #test_and_visualize_model(model1, weight_path="autoencoder_psnr_quantized.pth", test_loader=test_loader, title= f"{ds['name']}Reconstructed (PSNR + Quant)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)


--- Dataset: Kodak ---
Training Kodak Quantized Autoencoder with PSNR Loss...
Epoch [1/10], Quantized PSNR Loss: 0.7432, Compression Ratio: 12.00
Epoch [2/10], Quantized PSNR Loss: 0.7409, Compression Ratio: 12.00
Epoch [3/10], Quantized PSNR Loss: 0.7407, Compression Ratio: 12.00
Epoch [4/10], Quantized PSNR Loss: 0.7398, Compression Ratio: 12.00
Epoch [5/10], Quantized PSNR Loss: 0.7400, Compression Ratio: 12.00
Epoch [6/10], Quantized PSNR Loss: 0.7377, Compression Ratio: 12.00
Epoch [7/10], Quantized PSNR Loss: 0.7364, Compression Ratio: 12.00
Epoch [8/10], Quantized PSNR Loss: 0.7338, Compression Ratio: 12.00
Epoch [9/10], Quantized PSNR Loss: 0.7305, Compression Ratio: 12.00
Epoch [10/10], Quantized PSNR Loss: 0.7251, Compression Ratio: 12.00
-----------------------------------------------
Testing Kodak Quantized Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.007482, PSNR: 21.26, SSIM: 0.5495


In [34]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start AutoencoderQuantized
    model1 = AutoencoderQuantized(quantization_levels=16).to(device)
    optimizer1 = torch.optim.Adam(model1.parameters(), lr=1e-3)

    # Training: SSIM Loss
    print(f"Training {ds['name']} Quantized Autoencoder with SSIM Loss...")
    train_model(model1, ssim_loss, optimizer1, train_loader, num_epochs=10, loss_name="Quantized SSIM Loss", save_path="autoencoder_ssim_quantized.pth")

    print(f"Testing {ds['name']} Quantized Autoencoder...")
    #test_and_visualize_model(model1, weight_path="autoencoder_ssim_quantized.pth", test_loader=test_loader, title= f"{ds['name']}Reconstructed (SSIM + Quant)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)






--- Dataset: Kodak ---
Training Kodak Quantized Autoencoder with SSIM Loss...
Epoch [1/10], Quantized SSIM Loss: 0.5782, Compression Ratio: 12.00
Epoch [2/10], Quantized SSIM Loss: 0.5762, Compression Ratio: 12.00
Epoch [3/10], Quantized SSIM Loss: 0.5754, Compression Ratio: 12.00
Epoch [4/10], Quantized SSIM Loss: 0.5748, Compression Ratio: 12.00
Epoch [5/10], Quantized SSIM Loss: 0.5738, Compression Ratio: 12.00
Epoch [6/10], Quantized SSIM Loss: 0.5725, Compression Ratio: 12.00
Epoch [7/10], Quantized SSIM Loss: 0.5709, Compression Ratio: 12.00
Epoch [8/10], Quantized SSIM Loss: 0.5685, Compression Ratio: 12.00
Epoch [9/10], Quantized SSIM Loss: 0.5652, Compression Ratio: 12.00
Epoch [10/10], Quantized SSIM Loss: 0.5607, Compression Ratio: 12.00
-----------------------------------------------
Testing Kodak Quantized Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.006751, PSNR: 21.71, SSIM: 0.5210


Training with Improved Autoencoder

In [35]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start AutoencoderQuantized
    model2 = ImprovedAutoencoder().to(device)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)

    # Training: MSE Loss
    print(f"Training {ds['name']} Improved Autoencoder with MSE Loss...")
    train_model(model2, mse_loss, optimizer2, train_loader, num_epochs=10, loss_name="Improved MSE Loss", save_path="autoencoder_mse_impr.pth")

    print(f"Testing {ds['name']} Improved Autoencoder...")
    #test_and_visualize_model(model2, weight_path="autoencoder_mse_impr.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (MSE + Impr)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)


--- Dataset: Kodak ---
Training Kodak Improved Autoencoder with MSE Loss...
Epoch [1/10], Improved MSE Loss: 0.0407, Compression Ratio: 6.00
Epoch [2/10], Improved MSE Loss: 0.0193, Compression Ratio: 6.00
Epoch [3/10], Improved MSE Loss: 0.0148, Compression Ratio: 6.00
Epoch [4/10], Improved MSE Loss: 0.0151, Compression Ratio: 6.00
Epoch [5/10], Improved MSE Loss: 0.0128, Compression Ratio: 6.00
Epoch [6/10], Improved MSE Loss: 0.0122, Compression Ratio: 6.00
Epoch [7/10], Improved MSE Loss: 0.0126, Compression Ratio: 6.00
Epoch [8/10], Improved MSE Loss: 0.0117, Compression Ratio: 6.00
Epoch [9/10], Improved MSE Loss: 0.0098, Compression Ratio: 6.00
Epoch [10/10], Improved MSE Loss: 0.0086, Compression Ratio: 6.00
-----------------------------------------------
Testing Kodak Improved Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.007161, PSNR: 21.45, SSIM: 0.5789


In [36]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start ImprovedAutoencoder
    model2 = ImprovedAutoencoder().to(device)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)

    # Training: PSNR Loss
    print(f"Training {ds['name']} Improved Autoencoder with PSNR Loss...")
    train_model(model2, psnr_loss, optimizer2, train_loader, num_epochs=10, loss_name="Improved PSNR Loss", save_path="autoencoder_psnr_impr.pth")

    print(f"Testing {ds['name']} Improved Autoencoder...")
    #test_and_visualize_model(model2, weight_path="autoencoder_psnr_impr.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (PSNR + Impr)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)


--- Dataset: Kodak ---
Training Kodak Improved Autoencoder with PSNR Loss...
Epoch [1/10], Improved PSNR Loss: 0.7091, Compression Ratio: 6.00
Epoch [2/10], Improved PSNR Loss: 0.6369, Compression Ratio: 6.00
Epoch [3/10], Improved PSNR Loss: 0.6234, Compression Ratio: 6.00
Epoch [4/10], Improved PSNR Loss: 0.6087, Compression Ratio: 6.00
Epoch [5/10], Improved PSNR Loss: 0.6110, Compression Ratio: 6.00
Epoch [6/10], Improved PSNR Loss: 0.6081, Compression Ratio: 6.00
Epoch [7/10], Improved PSNR Loss: 0.5855, Compression Ratio: 6.00
Epoch [8/10], Improved PSNR Loss: 0.5992, Compression Ratio: 6.00
Epoch [9/10], Improved PSNR Loss: 0.5875, Compression Ratio: 6.00
Epoch [10/10], Improved PSNR Loss: 0.5739, Compression Ratio: 6.00
-----------------------------------------------
Testing Kodak Improved Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.006410, PSNR: 21.93, SSIM: 0.5365


In [37]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start ImprovedAutoencoder
    model2 = ImprovedAutoencoder().to(device)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)

    # Training: SSIM Loss
    print(f"Training {ds['name']} Improved Autoencoder with SSIM Loss...")
    train_model(model2, ssim_loss, optimizer2, train_loader, num_epochs=10, loss_name="Improved SSIM Loss", save_path="autoencoder_ssim_impr.pth")

    print(f"Testing {ds['name']} Improved Autoencoder...")
    #test_and_visualize_model(model2, weight_path="autoencoder_ssim_impr.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (SSIM + Impr)", output_color_space="rgb")


    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)


--- Dataset: Kodak ---
Training Kodak Improved Autoencoder with SSIM Loss...
Epoch [1/10], Improved SSIM Loss: 0.7956, Compression Ratio: 6.00
Epoch [2/10], Improved SSIM Loss: 0.6478, Compression Ratio: 6.00
Epoch [3/10], Improved SSIM Loss: 0.6135, Compression Ratio: 6.00
Epoch [4/10], Improved SSIM Loss: 0.5917, Compression Ratio: 6.00
Epoch [5/10], Improved SSIM Loss: 0.5839, Compression Ratio: 6.00
Epoch [6/10], Improved SSIM Loss: 0.5786, Compression Ratio: 6.00
Epoch [7/10], Improved SSIM Loss: 0.5716, Compression Ratio: 6.00
Epoch [8/10], Improved SSIM Loss: 0.5726, Compression Ratio: 6.00
Epoch [9/10], Improved SSIM Loss: 0.5716, Compression Ratio: 6.00
Epoch [10/10], Improved SSIM Loss: 0.5657, Compression Ratio: 6.00
-----------------------------------------------
Testing Kodak Improved Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.006263, PSNR: 22.03, SSIM: 0.5635


In [38]:
for ds in datasets_info:
    print(f"\n--- Dataset: {ds['name']} ---")
    train_loader = ds["train_loader"]
    test_loader = ds["test_loader"]

    # Start ImprovedAutoencoder
    model2 = ImprovedAutoencoder().to(device)
    optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-3)

    # Training: Combined Loss
    print(f"Training {ds['name']} Improved Autoencoder with Combined SSIM Loss...")
    train_model(model2, combined_loss, optimizer2, train_loader, num_epochs=10, loss_name="Improved Combined SSIM Loss", save_path="autoencoder_mse_ssim_impr.pth")

    print(f"Testing {ds['name']} Improved Autoencoder...")
    #test_and_visualize_model(model2, weight_path="autoencoder_mse_ssim_impr.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (MSE + SSIM + Impr)", output_color_space="rgb")

    print(f"JPEG compression comparison for {ds['name']} dataset:")
    test_jpeg_quality(test_loader, quality=2)


--- Dataset: Kodak ---
Training Kodak Improved Autoencoder with Combined SSIM Loss...
Epoch [1/10], Improved Combined SSIM Loss: 0.4267, Compression Ratio: 6.00
Epoch [2/10], Improved Combined SSIM Loss: 0.3333, Compression Ratio: 6.00
Epoch [3/10], Improved Combined SSIM Loss: 0.3286, Compression Ratio: 6.00
Epoch [4/10], Improved Combined SSIM Loss: 0.3223, Compression Ratio: 6.00
Epoch [5/10], Improved Combined SSIM Loss: 0.3204, Compression Ratio: 6.00
Epoch [6/10], Improved Combined SSIM Loss: 0.3207, Compression Ratio: 6.00
Epoch [7/10], Improved Combined SSIM Loss: 0.3171, Compression Ratio: 6.00
Epoch [8/10], Improved Combined SSIM Loss: 0.3165, Compression Ratio: 6.00
Epoch [9/10], Improved Combined SSIM Loss: 0.3134, Compression Ratio: 6.00
Epoch [10/10], Improved Combined SSIM Loss: 0.3106, Compression Ratio: 6.00
-----------------------------------------------
Testing Kodak Improved Autoencoder...
JPEG compression comparison for Kodak dataset:
JPEG Quality 2 - MSE: 0.00745

In [40]:
model2 = ImprovedAutoencoder().to(device)
optimizer2 = torch.optim.Adam(model2.parameters(), lr=1e-4)



print(f"Training Improved Autoencoder with Perceptual Loss...")
perc_train_model(model2, kodak_loader1, optimizer2, num_epochs=30, device="cpu")



Training Improved Autoencoder with Perceptual Loss...


RuntimeError: DataLoader worker (pid(s) 9824, 8528, 17424, 6164) exited unexpectedly

In [41]:
print("Testing Improved Autoencoder...")
#test_and_visualize_model(model2, weight_path="autoencoder_perceptual_impr.pth", test_loader=test_loader, title=f"{ds['name']} Reconstructed (Perceptual Loss)", output_color_space="rgb")
reconstruct_and_visualize(model2, kodak_loader1, device="cuda", num_images=24, title="Reconstruction VGG/PL")

print("JPEG compression comparison for dataset:")
test_jpeg_quality(test_loader, quality=2)


Testing Improved Autoencoder...


AssertionError: Torch not compiled with CUDA enabled